In [ ]:
# 2025/7/25
# zhangzhong
# https://docs.pytorch.org/docs/stable/distributed.html
# https://docs.pytorch.org/tutorials/intermediate/dist_tuto.html#collective-communication
# https://docs.pytorch.org/docs/stable/distributed.html#torch.distributed.gather_object

In [ ]:
# 当然可以！我们用一个最简单的例子来直观理解 torch.distributed.all_gather_object。

# ⸻

# 🧠 一句话理解

# all_gather_object 可以让每个进程收集所有其他进程发来的 Python 对象，放在一个列表里。

# 这个函数适合于非 Tensor 的 Python 对象，比如 dict、str、list、None、bool，你不需要手动做张量拼接或序列化，它会自动帮你完成。

# ⸻

# 🧪 实验环境准备（假设 2 个进程）

# 我们模拟 2 个进程（rank 0 和 rank 1），每个进程有自己的本地对象 local_data，我们想让两个进程都获得对方的内容。

# ⸻

# ✅ 示例代码：2个进程收集彼此的数据

# # dist_example.py

# import torch
# import torch.distributed as dist
# import os

# def setup(rank, world_size):
#     os.environ["MASTER_ADDR"] = "localhost"
#     os.environ["MASTER_PORT"] = "12355"
#     dist.init_process_group("gloo", rank=rank, world_size=world_size)

# def cleanup():
#     dist.destroy_process_group()

# def run(rank, world_size):
#     setup(rank, world_size)

#     # 每个进程有一个本地对象（任何 Python 对象都行）
#     local_data = {"rank": rank, "value": rank * 10}

#     # 创建用于收集的空列表
#     gathered_data = [None for _ in range(world_size)]

#     # 所有进程都调用 all_gather_object，同步收集所有人的 local_data
#     dist.all_gather_object(gathered_data, local_data)

#     print(f"[Rank {rank}] gathered_data: {gathered_data}")

#     cleanup()

# if __name__ == "__main__":
#     import multiprocessing as mp

#     world_size = 2
#     mp.spawn(run, args=(world_size,), nprocs=world_size)


# ⸻

# 🧾 输出结果：

# [Rank 0] gathered_data: [{'rank': 0, 'value': 0}, {'rank': 1, 'value': 10}]
# [Rank 1] gathered_data: [{'rank': 0, 'value': 0}, {'rank': 1, 'value': 10}]

# 也就是说：
# 	•	每个进程都收到了所有 rank 的对象。
# 	•	数据顺序和 rank 一致，gathered_data[0] 是 rank 0 的数据，gathered_data[1] 是 rank 1 的数据。

# ⸻

# 🧠 原理图解

# Rank	local_data	gathered_data after all_gather_object
# 0	{"rank": 0, "value": 0}	[{"rank": 0, "value": 0}, {"rank": 1, "value": 10}]
# 1	{"rank": 1, "value": 10}	[{"rank": 0, "value": 0}, {"rank": 1, "value": 10}]

# 底层原理是：PyTorch 会自动将 Python 对象序列化（pickle），广播给所有进程，并写入 gathered_data 对应的列表位置。

# ⸻

# 📌 和 all_gather 的区别？

# 函数	用于对象类型	例子
# all_gather	Tensor	所有 rank 上收集 tensor
# all_gather_object	任意 Python 对象	dict、str、list、None 等


# ⸻

# ✅ 总结

# 你有这个问题	用这个函数
# 想让所有进程收集彼此的非张量数据（如 dataloader.state_dict()）	all_gather_object
# 想收集 tensor 张量	all_gather


# ⸻

# 如果你想我再举一个例子，比如收集字符串、布尔值、list 或其他对象，我也可以写。你想要哪种？

In [ ]:
# 非常好的问题！

# 你说的这个需求——“只让其他进程把自己的数据发给 rank 0”——其实不应该用 all_gather_object，而是用：

# ✅ torch.distributed.gather_object()

# ⸻

# ✅ 正确用法：gather_object（只收集到 rank 0）

# 📌 特性总结：

# 特性	值
# 谁收集？	只有 dst（一般为 rank 0） 会收到所有进程发来的对象
# 谁发送？	所有进程都发送自己的对象（包含 rank 0 自己）
# 用途	只集中收集一次，用于保存 checkpoint 或调试汇总


# ⸻

# 🧪 最小例子：只把所有数据传给 rank 0

# import torch
# import torch.distributed as dist
# import os

# def setup(rank, world_size):
#     os.environ["MASTER_ADDR"] = "localhost"
#     os.environ["MASTER_PORT"] = "12355"
#     dist.init_process_group("gloo", rank=rank, world_size=world_size)

# def cleanup():
#     dist.destroy_process_group()

# def run(rank, world_size):
#     setup(rank, world_size)

#     local_data = {"rank": rank, "value": rank * 100}

#     # rank 0 会收集所有人的数据，其它 rank 填 None
#     gathered = None
#     if rank == 0:
#         gathered = [None for _ in range(world_size)]

#     dist.gather_object(local_data, gathered, dst=0)

#     if rank == 0:
#         print(f"[Rank 0] Gathered all data: {gathered}")

#     cleanup()

# if __name__ == "__main__":
#     import multiprocessing as mp
#     world_size = 4
#     mp.spawn(run, args=(world_size,), nprocs=world_size)


# ⸻

# 🧾 输出（仅 rank 0 打印）：

# [Rank 0] Gathered all data: [{'rank': 0, 'value': 0}, {'rank': 1, 'value': 100}, {'rank': 2, 'value': 200}, {'rank': 3, 'value': 300}]


# ⸻

# ✅ 用在你的场景里（只让 rank 0 保存 dataloader 状态）

# 你可以这样改你的代码：

# local_loader_state = dataloader.state_dict()
# gathered_loader_states = [None for _ in range(world_size)] if rank == 0 else None

# dist.gather_object(local_loader_state, gathered_loader_states, dst=0)

# if rank == 0:
#     torch.save({
#         "model": model.state_dict(),
#         "optimizer": optimizer.state_dict(),
#         "scaler": scaler.state_dict(),
#         "dataloader_states": gathered_loader_states,
#     }, "checkpoint.pt")


# ⸻

# 📌 你要记住的区别：

# 方法	所有进程都得到？	常用于
# all_gather_object	✅ 是	所有进程都需要这些数据
# gather_object	❌ 只有 rank 0 得到	集中保存、日志汇总、checkpoints


# ⸻

# # 如果你想我帮你封装一个 gather_and_save_checkpoint() 工具函数也可以说，我可以直接生成完整代码。是否继续？